In [1]:
import requests
import numpy as np
import pandas as pd
import datetime
from dotenv import load_dotenv

In [2]:
load_dotenv('.env')
fmp_key = os.getenv('FMP_API')

type(fmp_key)

str

In [3]:
company = 'AMD'

inc_stmt = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{company}?apikey={fmp_key}').json()
bal_sht = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?apikey={fmp_key}').json()
bal_sht

[{'date': '2020-12-26',
  'symbol': 'AMD',
  'reportedCurrency': 'USD',
  'fillingDate': '2021-01-29',
  'acceptedDate': '2021-01-29 17:24:00',
  'period': 'FY',
  'cashAndCashEquivalents': 1595000000,
  'shortTermInvestments': 695000000,
  'cashAndShortTermInvestments': 2290000000,
  'netReceivables': 2066000000,
  'inventory': 1399000000,
  'otherCurrentAssets': 388000000,
  'totalCurrentAssets': 6143000000,
  'propertyPlantEquipmentNet': 641000000,
  'goodwill': 289000000,
  'intangibleAssets': 0,
  'goodwillAndIntangibleAssets': 289000000,
  'longTermInvestments': 63000000,
  'taxAssets': 0,
  'otherNonCurrentAssets': 0,
  'totalNonCurrentAssets': 930000000,
  'otherAssets': 1889000000,
  'totalAssets': 8962000000,
  'accountPayables': 468000000,
  'shortTermDebt': 0,
  'taxPayables': 0,
  'deferredRevenue': 0,
  'otherCurrentLiabilities': 1949000000,
  'totalCurrentLiabilities': 2417000000,
  'longTermDebt': 330000000,
  'deferredRevenueNonCurrent': 0,
  'deferredTaxLiabilitiesNon

In [4]:
inc_stmt

[{'date': '2020-12-26',
  'symbol': 'AMD',
  'reportedCurrency': 'USD',
  'fillingDate': '2021-01-29',
  'acceptedDate': '2021-01-29 17:24:00',
  'period': 'FY',
  'revenue': 9763000000,
  'costOfRevenue': 5416000000,
  'grossProfit': 4347000000,
  'grossProfitRatio': 0.44525248386766364,
  'researchAndDevelopmentExpenses': 1983000000,
  'generalAndAdministrativeExpenses': 995000000,
  'sellingAndMarketingExpenses': 0,
  'otherExpenses': -47000000,
  'operatingExpenses': 2978000000,
  'costAndExpenses': 8394000000,
  'interestExpense': 47000000,
  'depreciationAndAmortization': 312000000,
  'ebitda': 1681000000,
  'ebitdaratio': 0.1721806821673666,
  'operatingIncome': 1369000000,
  'operatingIncomeRatio': 0.14022329202089523,
  'totalOtherIncomeExpensesNet': 0,
  'incomeBeforeTax': 1275000000,
  'incomeBeforeTaxRatio': 0.1305951039639455,
  'incomeTaxExpense': -1210000000,
  'netIncome': 2490000000,
  'netIncomeRatio': 0.25504455597664655,
  'eps': 2.1030405405405403,
  'epsdiluted': 

In [5]:
avg_rev_growth = ((1+(inc_stmt[0]['revenue'] - inc_stmt[1]['revenue']) / inc_stmt[1]['revenue']) * (1+(inc_stmt[1]['revenue'] - inc_stmt[2]['revenue']) / inc_stmt[2]['revenue']) * (1+(inc_stmt[2]['revenue'] - inc_stmt[3]['revenue']) / inc_stmt[3]['revenue']) * (1+(inc_stmt[3]['revenue'] - inc_stmt[4]['revenue']) / inc_stmt[4]['revenue']))**(1/4) -1

net_income = inc_stmt[0]['netIncome']

avg_rev_growth

0.22952683348150504

In [6]:
current_inc_stmt_df = pd.DataFrame.from_dict(inc_stmt[0], orient='index')
current_inc_stmt_df = current_inc_stmt_df[6:27]
current_inc_stmt_df.columns = ['2020 Current']
current_inc_stmt_df['Pct of Revenue'] = current_inc_stmt_df / current_inc_stmt_df.iloc[0]
current_inc_stmt_df['2019'] = pd.DataFrame.from_dict(inc_stmt[1], orient='index')
current_inc_stmt_df['2018'] = pd.DataFrame.from_dict(inc_stmt[2], orient='index')
current_inc_stmt_df['2017'] = pd.DataFrame.from_dict(inc_stmt[3], orient='index')
current_inc_stmt_df['2016'] = pd.DataFrame.from_dict(inc_stmt[4], orient='index')
complete_inc_stmt_df = current_inc_stmt_df[['2016', '2017', '2018', '2019', '2020 Current', 'Pct of Revenue']]
complete_inc_stmt_df 

,2016,2017,2018,2019,2020 Current,Pct of Revenue
revenue,4272000000,5253000000,6475000000,6731000000,9763000000,1
costOfRevenue,3274000000,3466000000,4028000000,3863000000,5416000000,0.554748
grossProfit,998000000,1787000000,2447000000,2868000000,4347000000,0.445252
grossProfitRatio,0.233614,0.340187,0.377915,0.426088,0.445252,4.56061e-11
researchAndDevelopmentExpenses,1008000000,1196000000,1434000000,1547000000,1983000000,0.203114
generalAndAdministrativeExpenses,460000000,516000000,562000000,750000000,995000000,0.101915
sellingAndMarketingExpenses,0,0,0,0,0,0
otherExpenses,80000000,-9000000,0,-165000000,-47000000,-0.00481409
operatingExpenses,1380000000,1672000000,2008000000,2413000000,2978000000,0.305029
costAndExpenses,4654000000,5138000000,6036000000,6276000000,8394000000,0.859777


In [7]:
complete_inc_stmt_df['2021 est'] = (complete_inc_stmt_df['2020 Current']['revenue'] * (1+avg_rev_growth)) * complete_inc_stmt_df['Pct of Revenue']
complete_inc_stmt_df['2022 est'] = (complete_inc_stmt_df['2021 est']['revenue'] * (1+avg_rev_growth)) * complete_inc_stmt_df['Pct of Revenue']
complete_inc_stmt_df['2023 est'] = (complete_inc_stmt_df['2022 est']['revenue'] * (1+avg_rev_growth)) * complete_inc_stmt_df['Pct of Revenue']
complete_inc_stmt_df['2024 est'] = (complete_inc_stmt_df['2023 est']['revenue'] * (1+avg_rev_growth)) * complete_inc_stmt_df['Pct of Revenue']
complete_inc_stmt_df['2025 est'] = (complete_inc_stmt_df['2024 est']['revenue'] * (1+avg_rev_growth)) * complete_inc_stmt_df['Pct of Revenue']
complete_inc_stmt_df

ipykernel_launcher:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
ipykernel_launcher:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,2016,2017,2018,2019,2020 Current,Pct of Revenue,2021 est,2022 est,2023 est,2024 est,2025 est
revenue,4272000000,5253000000,6475000000,6731000000,9763000000,1,1.20039e+10,1.47591e+10,1.81467e+10,2.23118e+10,2.7433e+10
costOfRevenue,3274000000,3466000000,4028000000,3863000000,5416000000,0.554748,6.65912e+09,8.18756e+09,1.00668e+10,1.23774e+10,1.52184e+10
grossProfit,998000000,1787000000,2447000000,2868000000,4347000000,0.445252,5.34475e+09,6.57152e+09,8.07986e+09,9.9344e+09,1.22146e+10
grossProfitRatio,0.233614,0.340187,0.377915,0.426088,0.445252,4.56061e-11,0.54745,0.673104,0.8276,1.01756,1.25111
researchAndDevelopmentExpenses,1008000000,1196000000,1434000000,1547000000,1983000000,0.203114,2.43815e+09,2.99777e+09,3.68584e+09,4.53184e+09,5.57202e+09
generalAndAdministrativeExpenses,460000000,516000000,562000000,750000000,995000000,0.101915,1.22338e+09,1.50418e+09,1.84943e+09,2.27392e+09,2.79585e+09
sellingAndMarketingExpenses,0,0,0,0,0,0,0,0,0,0,0
otherExpenses,80000000,-9000000,0,-165000000,-47000000,-0.00481409,-5.77878e+07,-7.10516e+07,-8.73599e+07,-1.07411e+08,-1.32065e+08
operatingExpenses,1380000000,1672000000,2008000000,2413000000,2978000000,0.305029,3.66153e+09,4.50195e+09,5.53527e+09,6.80576e+09,8.36787e+09
costAndExpenses,4654000000,5138000000,6036000000,6276000000,8394000000,0.859777,1.03206e+10,1.26895e+10,1.56021e+10,1.91832e+10,2.35863e+10


In [8]:
current_bal_sht_df = pd.DataFrame.from_dict(bal_sht[0],orient='index')
current_bal_sht_df = current_bal_sht_df[6:-2]
current_bal_sht_df.columns = ['2020 Current']
current_bal_sht_df['Pct of Revenue'] = current_bal_sht_df / complete_inc_stmt_df['2020 Current'].iloc[0]
current_bal_sht_df['2019'] = pd.DataFrame.from_dict(bal_sht[1],orient='index')
current_bal_sht_df['2018'] = pd.DataFrame.from_dict(bal_sht[2],orient='index')
current_bal_sht_df['2017'] = pd.DataFrame.from_dict(bal_sht[3],orient='index')
current_bal_sht_df['2016'] = pd.DataFrame.from_dict(bal_sht[4],orient='index')
complete_bal_sht_df = current_bal_sht_df[['2016', '2017', '2018', '2019', '2020 Current', 'Pct of Revenue']]
complete_bal_sht_df

,2016,2017,2018,2019,2020 Current,Pct of Revenue
cashAndCashEquivalents,1264000000,1185000000,1078000000,1466000000,1595000000,0.163372
shortTermInvestments,0,0,78000000,37000000,695000000,0.0711871
cashAndShortTermInvestments,1264000000,1185000000,1156000000,1503000000,2290000000,0.234559
netReceivables,311000000,454000000,1235000000,1859000000,2066000000,0.211615
inventory,751000000,694000000,845000000,982000000,1399000000,0.143296
otherCurrentAssets,109000000,301000000,304000000,253000000,388000000,0.0397419
totalCurrentAssets,2530000000,2634000000,3540000000,4597000000,6143000000,0.629212
propertyPlantEquipmentNet,164000000,261000000,348000000,500000000,641000000,0.065656
goodwill,289000000,289000000,289000000,289000000,289000000,0.0296016
intangibleAssets,232000000,239000000,226000000,210000000,0,0


In [9]:
complete_bal_sht_df['2021 est'] = complete_inc_stmt_df['2021 est']['revenue'] * complete_bal_sht_df['Pct of Revenue']
complete_bal_sht_df['2022 est'] = complete_inc_stmt_df['2022 est']['revenue'] * complete_bal_sht_df['Pct of Revenue']
complete_bal_sht_df['2023 est'] = complete_inc_stmt_df['2023 est']['revenue'] * complete_bal_sht_df['Pct of Revenue']
complete_bal_sht_df['2024 est'] = complete_inc_stmt_df['2024 est']['revenue'] * complete_bal_sht_df['Pct of Revenue']
complete_bal_sht_df['2025 est'] = complete_inc_stmt_df['2025 est']['revenue'] * complete_bal_sht_df['Pct of Revenue']
complete_bal_sht_df 

ipykernel_launcher:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
ipykernel_launcher:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,2016,2017,2018,2019,2020 Current,Pct of Revenue,2021 est,2022 est,2023 est,2024 est,2025 est
cashAndCashEquivalents,1264000000,1185000000,1078000000,1466000000,1595000000,0.163372,1.9611e+09,2.41122e+09,2.96466e+09,3.64513e+09,4.48178e+09
shortTermInvestments,0,0,78000000,37000000,695000000,0.0711871,8.54521e+08,1.05066e+09,1.29181e+09,1.58832e+09,1.95288e+09
cashAndShortTermInvestments,1264000000,1185000000,1156000000,1503000000,2290000000,0.234559,2.81562e+09,3.46188e+09,4.25647e+09,5.23344e+09,6.43466e+09
netReceivables,311000000,454000000,1235000000,1859000000,2066000000,0.211615,2.5402e+09,3.12325e+09,3.84012e+09,4.72153e+09,5.80524e+09
inventory,751000000,694000000,845000000,982000000,1399000000,0.143296,1.72011e+09,2.11492e+09,2.60035e+09,3.1972e+09,3.93104e+09
otherCurrentAssets,109000000,301000000,304000000,253000000,388000000,0.0397419,4.77056e+08,5.86554e+08,7.21183e+08,8.86714e+08,1.09024e+09
totalCurrentAssets,2530000000,2634000000,3540000000,4597000000,6143000000,0.629212,7.55298e+09,9.2866e+09,1.14181e+10,1.40389e+10,1.72612e+10
propertyPlantEquipmentNet,164000000,261000000,348000000,500000000,641000000,0.065656,7.88127e+08,9.69023e+08,1.19144e+09,1.46491e+09,1.80114e+09
goodwill,289000000,289000000,289000000,289000000,289000000,0.0296016,3.55333e+08,4.36892e+08,5.3717e+08,6.60465e+08,8.1206e+08
intangibleAssets,232000000,239000000,226000000,210000000,0,0,0,0,0,0,0


In [10]:
financial_analysis = {}


financial_analysis['Liquidity Ratios'] = {}


financial_analysis['Liquidity Ratios']['Current Ratio'] = complete_bal_sht_df['2020 Current']['totalCurrentAssets'] / complete_bal_sht_df['2020 Current']['totalCurrentLiabilities']

financial_analysis['Liquidity Ratios']['Quick Ratio'] = (complete_bal_sht_df['2020 Current']['totalCurrentAssets'] - complete_bal_sht_df['2020 Current']['inventory'] - complete_bal_sht_df['2020 Current']['otherCurrentAssets'])/ complete_bal_sht_df['2020 Current']['totalCurrentLiabilities']

financial_analysis['Liquidity Ratios']['Cash Ratio'] = (complete_bal_sht_df['2020 Current']['totalCurrentAssets'] - complete_bal_sht_df['2020 Current']['inventory'] - complete_bal_sht_df['2020 Current']['netReceivables'] - complete_bal_sht_df['2020 Current']['otherCurrentAssets']) / complete_bal_sht_df['2020 Current']['totalCurrentLiabilities']


financial_analysis['Solvency Ratios'] = {}


financial_analysis['Solvency Ratios']['Debt-Equity'] = complete_bal_sht_df['2020 Current']['totalLiabilities'] / complete_bal_sht_df['2020 Current']['totalStockholdersEquity']

financial_analysis['Solvency Ratios']['Debt-Capital'] = complete_bal_sht_df['2020 Current']['totalLiabilities'] / complete_bal_sht_df['2020 Current']['totalLiabilitiesAndStockholdersEquity']


financial_analysis['Profitability Ratios'] = {}


financial_analysis['Profitability Ratios']['Profit Margin'] = complete_inc_stmt_df['2020 Current']['netIncome'] / complete_inc_stmt_df['2020 Current']['revenue']

financial_analysis['Profitability Ratios']['Operating Margin'] = complete_inc_stmt_df['2020 Current']['operatingIncomeRatio']

financial_analysis['Profitability Ratios']['Gross Margin'] = complete_inc_stmt_df['2020 Current']['grossProfitRatio']

financial_analysis['Profitability Ratios']['ROA'] = complete_inc_stmt_df['2020 Current']['netIncome'] / ((complete_bal_sht_df['2020 Current']['totalAssets'] + complete_bal_sht_df['2019']['totalAssets'])/2)

financial_analysis['Profitability Ratios']['ROE'] = complete_inc_stmt_df['2020 Current']['netIncome'] / ((complete_bal_sht_df['2020 Current']['totalStockholdersEquity'] + complete_bal_sht_df['2019']['totalStockholdersEquity'])/2)


financial_analysis['Debt Coverage Ratios'] = {}


financial_analysis['Debt Coverage Ratios']['Interest Coverage'] = complete_inc_stmt_df['2020 Current']['incomeBeforeTax'] / complete_inc_stmt_df['2020 Current']['interestExpense']

financial_analysis['Debt Coverage Ratios']['Debt Service Coverage'] =complete_inc_stmt_df['2020 Current']['incomeBeforeTax'] / complete_bal_sht_df['2020 Current']['totalLiabilities']

financial_analysis['Debt Coverage Ratios']['Asset Coverage'] =  (complete_bal_sht_df['2020 Current']['totalAssets'] - complete_bal_sht_df['2020 Current']['totalCurrentLiabilities'])/ complete_bal_sht_df['2020 Current']['totalCurrentLiabilities']



financial_analysis


{'Liquidity Ratios': {'Current Ratio': 2.5415804716590813,
  'Quick Ratio': 1.8022341745966073,
  'Cash Ratio': 0.9474555233760861},
 'Solvency Ratios': {'Debt-Equity': 0.5353777625492547,
  'Debt-Capital': 0.34869448783753626},
 'Profitability Ratios': {'Profit Margin': 0.25504455597664655,
  'Operating Margin': 0.14022329202089523,
  'Gross Margin': 0.44525248386766364,
  'ROA': 0.3322214809873249,
  'ROE': 0.574792243767313},
 'Debt Coverage Ratios': {'Interest Coverage': 27.127659574468087,
  'Debt Service Coverage': 0.408,
  'Asset Coverage': 2.7079023582954074}}

In [11]:
financial_analysis_df = pd.DataFrame.from_dict(financial_analysis, orient='columns')
pd.options.display.float_format = '{:,.2f}'.format
financial_analysis_df = financial_analysis_df.reindex(['Current Ratio', 'Quick Ratio', 'Cash Ratio','Debt-Equity', 'Debt-Capital', 'Profit Margin', 'Operating Margin', 'Gross Margin', 'ROA', 'ROE', 'Asset Coverage', 'Interest Coverage', 'Debt Service Coverage'])

liquidity_analysis = financial_analysis_df['Liquidity Ratios'].dropna()
solvency_analysis = financial_analysis_df['Solvency Ratios'].dropna()
profitability_analysis = financial_analysis_df['Profitability Ratios'].dropna()
debt_coverage_analysis = financial_analysis_df['Debt Coverage Ratios'].dropna()

financial_analysis_df

,Liquidity Ratios,Solvency Ratios,Profitability Ratios,Debt Coverage Ratios
Current Ratio,2.54,nan,nan,nan
Quick Ratio,1.80,nan,nan,nan
Cash Ratio,0.95,nan,nan,nan
Debt-Equity,nan,0.54,nan,nan
Debt-Capital,nan,0.35,nan,nan
Profit Margin,nan,nan,0.26,nan
Operating Margin,nan,nan,0.14,nan
Gross Margin,nan,nan,0.45,nan
ROA,nan,nan,0.33,nan
ROE,nan,nan,0.57,nan


In [12]:
liquidity_analysis

Current Ratio   2.54
Quick Ratio     1.80
Cash Ratio      0.95
Name: Liquidity Ratios, dtype: float64

In [13]:
solvency_analysis

Debt-Equity    0.54
Debt-Capital   0.35
Name: Solvency Ratios, dtype: float64

In [14]:
profitability_analysis

Profit Margin      0.26
Operating Margin   0.14
Gross Margin       0.45
ROA                0.33
ROE                0.57
Name: Profitability Ratios, dtype: float64

In [15]:
debt_coverage_analysis

Asset Coverage           2.71
Interest Coverage       27.13
Debt Service Coverage    0.41
Name: Debt Coverage Ratios, dtype: float64

In [16]:
projected_cf = {}

projected_cf['2021 est'] = {}

projected_cf['2021 est']['Net Income'] = complete_inc_stmt_df['2021 est']['netIncome']
projected_cf['2021 est']['Depreciation & Amortization'] = complete_inc_stmt_df['2021 est']['depreciationAndAmortization']
projected_cf['2021 est']['Change in Receivables'] = complete_bal_sht_df['2020 Current']['netReceivables'] - complete_bal_sht_df['2021 est']['netReceivables']
projected_cf['2021 est']['Change in Inventory'] = complete_bal_sht_df['2020 Current']['inventory'] - complete_bal_sht_df['2021 est']['inventory']
projected_cf['2021 est']['Change in Payables'] = complete_bal_sht_df['2021 est']['accountPayables'] - complete_bal_sht_df['2020 Current']['accountPayables']
projected_cf['2021 est']['Change in NWC'] = projected_cf['2021 est']['Change in Receivables'] + projected_cf['2021 est']['Change in Inventory'] + projected_cf['2021 est']['Change in Payables']
projected_cf['2021 est']['OP CF'] = projected_cf['2021 est']['Net Income'] + projected_cf['2021 est']['Depreciation & Amortization'] + projected_cf['2021 est']['Change in NWC']
projected_cf['2021 est']['CAPEX'] = complete_bal_sht_df['2021 est']['propertyPlantEquipmentNet'] - complete_bal_sht_df['2020 Current']['propertyPlantEquipmentNet'] + complete_inc_stmt_df['2021 est']['depreciationAndAmortization']
projected_cf['2021 est']['Unlevered FCF'] = projected_cf['2021 est']['CAPEX'] + projected_cf['2021 est']['OP CF']
projected_cf['2021 est']['Total Debt'] = -(complete_bal_sht_df['2021 est']['totalCurrentLiabilities'])
projected_cf['2021 est']['Levered FCF'] = projected_cf['2021 est']['Unlevered FCF'] + projected_cf['2021 est']['Total Debt']


projected_cf['2022 est'] = {}

projected_cf['2022 est']['Net Income'] = complete_inc_stmt_df['2022 est']['netIncome']
projected_cf['2022 est']['Depreciation & Amortization'] = complete_inc_stmt_df['2022 est']['depreciationAndAmortization']
projected_cf['2022 est']['Change in Receivables'] = complete_bal_sht_df['2021 est']['netReceivables'] - complete_bal_sht_df['2022 est']['netReceivables']
projected_cf['2022 est']['Change in Inventory'] = complete_bal_sht_df['2021 est']['inventory'] - complete_bal_sht_df['2022 est']['inventory']
projected_cf['2022 est']['Change in Payables'] = complete_bal_sht_df['2022 est']['accountPayables'] - complete_bal_sht_df['2021 est']['accountPayables']
projected_cf['2022 est']['Change in NWC'] = projected_cf['2022 est']['Change in Receivables'] + projected_cf['2022 est']['Change in Inventory'] + projected_cf['2022 est']['Change in Payables']
projected_cf['2022 est']['OP CF'] = projected_cf['2022 est']['Net Income'] + projected_cf['2022 est']['Depreciation & Amortization'] + projected_cf['2022 est']['Change in NWC']
projected_cf['2022 est']['CAPEX'] = complete_bal_sht_df['2022 est']['propertyPlantEquipmentNet'] - complete_bal_sht_df['2021 est']['propertyPlantEquipmentNet'] + complete_inc_stmt_df['2022 est']['depreciationAndAmortization']
projected_cf['2022 est']['Unlevered FCF'] = projected_cf['2022 est']['CAPEX'] + projected_cf['2022 est']['OP CF']
projected_cf['2022 est']['Total Debt'] = -(complete_bal_sht_df['2022 est']['totalCurrentLiabilities'])
projected_cf['2022 est']['Levered FCF'] = projected_cf['2022 est']['Unlevered FCF'] + projected_cf['2022 est']['Total Debt']


projected_cf['2023 est'] = {}

projected_cf['2023 est']['Net Income'] = complete_inc_stmt_df['2023 est']['netIncome']
projected_cf['2023 est']['Depreciation & Amortization'] = complete_inc_stmt_df['2023 est']['depreciationAndAmortization']
projected_cf['2023 est']['Change in Receivables'] = complete_bal_sht_df['2022 est']['netReceivables'] - complete_bal_sht_df['2023 est']['netReceivables']
projected_cf['2023 est']['Change in Inventory'] = complete_bal_sht_df['2022 est']['inventory'] - complete_bal_sht_df['2023 est']['inventory']
projected_cf['2023 est']['Change in Payables'] = complete_bal_sht_df['2023 est']['accountPayables'] - complete_bal_sht_df['2022 est']['accountPayables']
projected_cf['2023 est']['Change in NWC'] = projected_cf['2023 est']['Change in Receivables'] + projected_cf['2023 est']['Change in Inventory'] + projected_cf['2023 est']['Change in Payables']
projected_cf['2023 est']['OP CF'] = projected_cf['2023 est']['Net Income'] + projected_cf['2023 est']['Depreciation & Amortization'] + projected_cf['2023 est']['Change in NWC']
projected_cf['2023 est']['CAPEX'] = complete_bal_sht_df['2023 est']['propertyPlantEquipmentNet'] - complete_bal_sht_df['2022 est']['propertyPlantEquipmentNet'] + complete_inc_stmt_df['2023 est']['depreciationAndAmortization']
projected_cf['2023 est']['Unlevered FCF'] = projected_cf['2023 est']['CAPEX'] + projected_cf['2023 est']['OP CF']
projected_cf['2023 est']['Total Debt'] = -(complete_bal_sht_df['2023 est']['totalCurrentLiabilities'])
projected_cf['2023 est']['Levered FCF'] = projected_cf['2023 est']['Unlevered FCF'] + projected_cf['2023 est']['Total Debt']


projected_cf['2024 est'] = {}

projected_cf['2024 est']['Net Income'] = complete_inc_stmt_df['2024 est']['netIncome']
projected_cf['2024 est']['Depreciation & Amortization'] = complete_inc_stmt_df['2024 est']['depreciationAndAmortization']
projected_cf['2024 est']['Change in Receivables'] = complete_bal_sht_df['2023 est']['netReceivables'] - complete_bal_sht_df['2024 est']['netReceivables']
projected_cf['2024 est']['Change in Inventory'] = complete_bal_sht_df['2023 est']['inventory'] - complete_bal_sht_df['2024 est']['inventory']
projected_cf['2024 est']['Change in Payables'] = complete_bal_sht_df['2024 est']['accountPayables'] - complete_bal_sht_df['2023 est']['accountPayables']
projected_cf['2024 est']['Change in NWC'] = projected_cf['2024 est']['Change in Receivables'] + projected_cf['2024 est']['Change in Inventory'] + projected_cf['2024 est']['Change in Payables']
projected_cf['2024 est']['OP CF'] = projected_cf['2024 est']['Net Income'] + projected_cf['2024 est']['Depreciation & Amortization'] + projected_cf['2024 est']['Change in NWC']
projected_cf['2024 est']['CAPEX'] = complete_bal_sht_df['2024 est']['propertyPlantEquipmentNet'] - complete_bal_sht_df['2023 est']['propertyPlantEquipmentNet'] + complete_inc_stmt_df['2024 est']['depreciationAndAmortization']
projected_cf['2024 est']['Unlevered FCF'] = projected_cf['2024 est']['CAPEX'] + projected_cf['2024 est']['OP CF']
projected_cf['2024 est']['Total Debt'] = -(complete_bal_sht_df['2024 est']['totalCurrentLiabilities'])
projected_cf['2024 est']['Levered FCF'] = projected_cf['2024 est']['Unlevered FCF'] + projected_cf['2024 est']['Total Debt']


projected_cf['2025 est'] = {}

projected_cf['2025 est']['Net Income'] = complete_inc_stmt_df['2025 est']['netIncome']
projected_cf['2025 est']['Depreciation & Amortization'] = complete_inc_stmt_df['2025 est']['depreciationAndAmortization']
projected_cf['2025 est']['Change in Receivables'] = complete_bal_sht_df['2024 est']['netReceivables'] - complete_bal_sht_df['2025 est']['netReceivables']
projected_cf['2025 est']['Change in Inventory'] = complete_bal_sht_df['2024 est']['inventory'] - complete_bal_sht_df['2025 est']['inventory']
projected_cf['2025 est']['Change in Payables'] = complete_bal_sht_df['2025 est']['accountPayables'] - complete_bal_sht_df['2024 est']['accountPayables']
projected_cf['2025 est']['Change in NWC'] = projected_cf['2025 est']['Change in Receivables'] + projected_cf['2025 est']['Change in Inventory'] + projected_cf['2025 est']['Change in Payables']
projected_cf['2025 est']['OP CF'] = projected_cf['2025 est']['Net Income'] + projected_cf['2025 est']['Depreciation & Amortization'] + projected_cf['2025 est']['Change in NWC']
projected_cf['2025 est']['CAPEX'] = complete_bal_sht_df['2025 est']['propertyPlantEquipmentNet'] - complete_bal_sht_df['2024 est']['propertyPlantEquipmentNet'] + complete_inc_stmt_df['2025 est']['depreciationAndAmortization']
projected_cf['2025 est']['Unlevered FCF'] = projected_cf['2025 est']['CAPEX'] + projected_cf['2025 est']['OP CF']
projected_cf['2025 est']['Total Debt'] = -(complete_bal_sht_df['2025 est']['totalCurrentLiabilities'])
projected_cf['2025 est']['Levered FCF'] = projected_cf['2025 est']['Unlevered FCF'] + projected_cf['2025 est']['Total Debt']

projected_cf

{'2021 est': {'Net Income': 3061521815.368948,
  'Depreciation & Amortization': 383612372.0462296,
  'Change in Receivables': -474202437.9727893,
  'Change in Inventory': -321108040.04062533,
  'Change in Payables': 107418558.06934428,
  'Change in NWC': -687891919.9440703,
  'OP CF': 2757242267.471107,
  'CAPEX': 530739072.3078743,
  'Unlevered FCF': 3287981339.778981,
  'Total Debt': -2971766356.524798,
  'Levered FCF': 316214983.2541833},
 '2022 est': {'Net Income': 3764223223.2851315,
  'Depreciation & Amortization': 471661705.0863297,
  'Change in Receivables': -583044621.9898939,
  'Change in Inventory': -394810951.67660284,
  'Change in Payables': 132073999.56015027,
  'Change in NWC': -845781574.1063465,
  'OP CF': 3390103354.2651143,
  'CAPEX': 652557930.9796122,
  'Unlevered FCF': 4042661285.2447267,
  'Total Debt': -3653866478.1848044,
  'Levered FCF': 388794807.0599222},
 '2023 est': {'Net Income': 4628213460.243312,
  'Depreciation & Amortization': 579920722.7292824,
  'Ch

In [17]:
cf_forecast = pd.DataFrame.from_dict(projected_cf, orient='columns')
pd.options.display.float_format = '{:,.0f}'.format
cf_forecast = cf_forecast.reindex(['Net Income', 'Depreciation & Amortization', 'Change in Receivables','Change in Inventory', 'Change in Payables', 'Change in NWC', 'OP CF', 'CAPEX', 'Unlevered FCF', 'Total Debt', 'Levered FCF'])

cf_forecast

,2021 est,2022 est,2023 est,2024 est,2025 est
Net Income,"3,061,521,815","3,764,223,223","4,628,213,460","5,690,512,640","6,996,637,988"
Depreciation & Amortization,"383,612,372","471,661,705","579,920,723","713,028,090","876,687,170"
Change in Receivables,"-474,202,438","-583,044,622","-716,869,008","-881,409,681","-1,083,716,854"
Change in Inventory,"-321,108,040","-394,810,952","-485,430,659","-596,850,021","-733,843,117"
Change in Payables,"107,418,558","132,074,000","162,388,526","199,661,051","245,488,619"
Change in NWC,"-687,891,920","-845,781,574","-1,039,911,141","-1,278,598,652","-1,572,071,352"
OP CF,"2,757,242,267","3,390,103,354","4,168,223,042","5,124,942,078","6,301,253,806"
CAPEX,"530,739,072","652,557,931","802,337,487","986,495,469","1,212,922,651"
Unlevered FCF,"3,287,981,340","4,042,661,285","4,970,560,529","6,111,437,548","7,514,176,456"
Total Debt,"-2,971,766,357","-3,653,866,478","-4,492,526,881","-5,523,682,350","-6,791,515,669"
